In [7]:
import torch, torchaudio
import requests
from io import BytesIO

from IPython.display import Audio

Load the Hubert content encoder (see https://github.com/bshall/hubert/)

In [4]:
hubert, encode = torch.hub.load("bshall/dusted:main", "hubert", language="english", trust_repo=True)
hubert = hubert.cuda()

Using cache found in /home/benjamin/.cache/torch/hub/bshall_dusted_main
Using cache found in /home/benjamin/.cache/torch/hub/bshall_hubert_main


Load the k-means checkpoint

In [5]:
kmeans, segment = torch.hub.load("bshall/dusted:main", "kmeans", language="english", trust_repo=True)

Using cache found in /home/benjamin/.cache/torch/hub/bshall_dusted_main
Downloading: "https://github.com/bshall/dusted/releases/download/v0.1/kmeans-english-50f36a.pt" to /home/benjamin/.cache/torch/hub/checkpoints/kmeans-english-50f36a.pt
100.0%


Load the similarity function and pattern matcher

In [27]:
sim, match = torch.hub.load("bshall/dusted:main", "dusted", trust_repo=True)

Using cache found in /home/benjamin/.cache/torch/hub/bshall_dusted_main


Load a pair of audio clips. To demonstrate, we'll download two excerpts from the book [Don Quixote](https://en.wikipedia.org/wiki/Don_Quixote) (originally from [LibriSpeech](https://www.openslr.org/12)).

The transcriptions are:
- Don Quixote did so, and asked him what had happened to him, and what he was afraid of.
- Don Quixote was on foot with his horse unbridled and his lance leaning against a tree, and in short completely defenceless; he thought it best therefore to fold his arms and bow his head and reserve himself for a more favourable occasion and opportunity.

In [16]:
response = requests.get("https://github.com/bshall/dusted/releases/download/v0.1/don-quixote-did-so.wav")
content = BytesIO(response.content)

xwav, _ = torchaudio.load(content)

response = requests.get("https://github.com/bshall/dusted/releases/download/v0.1/don-quixote-was-on-foot.wav")
content = BytesIO(response.content)

ywav, _ = torchaudio.load(content)

You can listen to the examples below:

In [17]:
Audio(xwav.squeeze(), rate=sr)

In [18]:
Audio(ywav.squeeze(), rate=sr)

Encode the audio using HuBERT:

In [19]:
xwav = xwav.unsqueeze(0).cuda()
ywav = ywav.unsqueeze(0).cuda()

x = encode(hubert, xwav).squeeze().cpu().numpy()
y = encode(hubert, ywav).squeeze().cpu().numpy()

Segment the features into phone-like units:

In [20]:
xcodes, xboundaries = segment(x, kmeans.cluster_centers_, gamma=0.2)
ycodes, yboundaries = segment(y, kmeans.cluster_centers_, gamma=0.2)

Search for recurring unit sub-sequences:

The `match` function uses `numba` so the first run can be slow, but the next call should be fast!

In [31]:
for path, a, b, similarity in match(xcodes, ycodes, sim, 1, threshold=6):
    # Find start and end times of the matching sub-sequences
    a0 = round(xboundaries[a0 - 1] * 0.02, 2)
    an = round(xboundaries[an] * 0.02, 2)
    b0 = round(yboundaries[b0 - 1] * 0.02, 2)
    bn = round(yboundaries[bn] * 0.02, 2)
    break